# Table of Contents
 <p><div class="lev1 toc-item"><a href="#PMT/ADC" data-toc-modified-id="PMT/ADC-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PMT/ADC</a></div><div class="lev2 toc-item"><a href="#Example-of-how-to-compress-bytes-(e.g.,-JSON)-to-bzip2" data-toc-modified-id="Example-of-how-to-compress-bytes-(e.g.,-JSON)-to-bzip2-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Example of how to compress bytes (e.g., JSON) to bzip2</a></div><div class="lev1 toc-item"><a href="#Pump" data-toc-modified-id="Pump-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pump</a></div>

# PMT/ADC

In [ ]:
import gtk
import gobject
import threading
import datetime as dt

import matplotlib as mpl
import matplotlib.style
import numpy as np
import pandas as pd

from streaming_plot import StreamingPlot


def _generate_data(stop_event, data_ready, data):
    delta_t = dt.timedelta(seconds=.1)
    samples_per_plot = 5

    while True:
        time_0 = dt.datetime.now()
        values_i = np.random.rand(samples_per_plot)
        absolute_times_i = pd.Series([time_0 + i * delta_t
                                      for i in xrange(len(values_i))])
        data_i = pd.Series(values_i, index=absolute_times_i)
        data.append(data_i)
        data_ready.set()
        if stop_event.wait(samples_per_plot *
                           delta_t.total_seconds()):
            break
        
with mpl.style.context('seaborn',
                       {'image.cmap': 'gray',
                        'image.interpolation' : 'none'}):
    win = gtk.Window()
    win.set_default_size(800, 600)
    view = StreamingPlot(data_func=_generate_data)
    win.add(view.widget)
    win.connect('check-resize', lambda *args: view.on_resize())
    win.set_position(gtk.WIN_POS_MOUSE)
    win.show_all()
    view.fig.tight_layout()
    win.connect('destroy', gtk.main_quit)
    gobject.idle_add(view.start)
    
    def auto_close(*args):
        if not view.stop_event.is_set():
            # User did not explicitly pause the measurement.  Automatically
            # close the measurement and continue.
            win.destroy()
    gobject.timeout_add(5000, auto_close)
    
    measurement_complete = threading.Event()
    
    view.widget.connect('destroy', lambda *args: measurement_complete.set())

    gtk.gdk.threads_init()
    gtk.gdk.threads_enter()
    gtk.main()
    gtk.gdk.threads_leave()
    
    print measurement_complete.wait()

## Example of how to compress bytes (e.g., JSON) to bzip2

In [ ]:
from IPython.display import display
import bz2


data = pd.concat(view.data)
data_json = data.to_json()
data_json_bz2 = bz2.compress(data_json)
data_from_json = pd.read_json(bz2.decompress(data_json_bz2), typ='series')
len(data_json), len(data_json_bz2)

------------------------------------------------------------------------

# Pump #

In [ ]:
import gobject
import gtk

from pump_gtk_ui import PumpControl


win = gtk.Window()
win.set_modal(True)
win.set_default_size(480, 150)

pump_control_view = PumpControl(None)
win.add(pump_control_view.widget)
win.set_position(gtk.WIN_POS_MOUSE)
win.show_all()
gobject.idle_add(pump_control_view.start)
win.connect('destroy', gtk.main_quit)
gtk.gdk.threads_init()
gtk.main()